In [3]:
import pandas as pd

# Load the data
df = pd.read_excel('C:\\Users\\user\\Downloads\\DirtyData.xlsx')
df.head() # Show first 5 observations
    
     


# Assuming the first two rows are not part of the data, and actual column names start from the third row
df.columns = df.iloc[1]  # Set the second row as the header
df = df.drop(df.index[0:2])  # Drop the first two rows which are not part of the data
    
     


# Correct the column names for clarity
df.columns = [
    'Year', 'Month', 'Brand', 'Model', 'Type', 'Color',
    'Transmission', 'Fuel Type', 'Price', 'Kilometers', 'Units Sold'
]

# Apply string operations to string columns
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype(str).str.strip().str.title()

# Specifically strip spaces and remove quotes from 'Model' and 'Transmission'
df['Model'] = df['Model'].str.strip().str.replace('"', '')
df['Transmission'] = df['Transmission'].str.strip().str.replace('"', '')
    
     


# Convert 'Year' to numeric type and handle non-numeric values
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

# Convert 'Price', 'Kilometers', and 'Units Sold' to numeric types and handle missing values
for col in ['Price', 'Kilometers', 'Units Sold']:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col].fillna(df[col].mean(), inplace=True)

# Replace negative values in 'Units Sold' with their absolute values
df['Units Sold'] = df['Units Sold'].apply(lambda x: abs(x) if x < 0 else x)
    
     


# Define a function to replace outliers with the mean
def replace_outliers_with_mean(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    mean_value = series.mean()
    return series.apply(lambda x: mean_value if x < lower_bound or x > upper_bound else x)

# Handle outliers in 'Price' and 'Kilometers'
df['Price'] = replace_outliers_with_mean(df['Price'])
df['Kilometers'] = replace_outliers_with_mean(df['Kilometers'])
    
     


# Create a date column from 'Year' and 'Month', if possible
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '-' + df['Month'], format='%Y-%b', errors='coerce')

# Save the cleaned data to a new Excel file
df.to_excel('C:\\Users\\user\\Downloads\\CleanedData.xlsx', index=False)

# Display the cleaned data
df.head()




,Year,Month,Brand,Model,Type,Color,Transmission,Fuel Type,Price,Kilometers,Units Sold,Date
2,2011,Jan,Ford,Fiesta,Sedan,Blue,Manual,Petrol,18000.0,15000.0,25.0,2011-01-01
3,2011,Jan,Vauxhall,Corsa,Hatchback,Red,Automatic,Diesel,20000.0,18000.0,22.0,2011-01-01
4,2011,Feb,Bmw,3 Series,Sedan,Black,Manual,Petrol,30000.0,20000.0,18.0,2011-02-01
5,2011,Feb,Volkswagen,Golf,Hatchback,White,Automatic,Petrol,25000.0,16000.0,20.0,2011-02-01
6,2011,Mar,Audi,A3,Hatchback,Silver,Manual,Diesel,28000.0,12000.0,28.0,2011-03-01
